In [1]:
import time
from design import *
import importlib
import shutil
from utils import *
from openai import OpenAI
from prompts import *
import json
import numpy as np
from gymnasium.envs.robodesign.GPTHopper import GPTHopperEnv
import os

In [ ]:
import prompts
class DGA:
    def __init__(self):
        self.client = OpenAI(api_key=api_key)
        # self.model = "gpt-3.5-turbo"
        self.model = "gpt-4-turbo"


    def extract_code(self, text):
        match = re.search(r'```python\n(.*?)\n```', text, re.DOTALL)
        return match.group(1).strip() if match else None

    def indent_code(self, code):
        return "\n".join(line if line.strip() else line for line in code.split("\n"))

    def generate_rewardfunc(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        responses = self.client.chat.completions.create(
            model=self.model, messages=messages, n=rewardfunc_nums
        )
        files = []
        for i, choice in enumerate(responses.choices):
            reward_code = self.extract_code(choice.message.content)
            if reward_code:
                full_code = self.indent_code(reward_code) + "\n"
                file_name =  f"GPTHopper_{i}.py"
                file_path = os.path.join(folder_name, "env", file_name)
                with open(file_path, "w") as fp:
                    fp.write(full_code)

                with open(file_path, "w") as fp:
                    fp.write(full_code)
                files.append(file_path)
                print(f"Saved: {file_path}")
        return files
    
    def generate_rewardfunc_div(self, rewardfunc_nums, folder_name):

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content": rewardfunc_prompts + zeroshot_rewardfunc_format}
        ]

        # 生成初始 Reward Function
        response = self.client.chat.completions.create(
            model=self.model, messages=messages, n=1, timeout=10
        )

        rewardfunc_files = []

        initial_code = self.extract_code(response.choices[0].message.content)
        if initial_code:
            reward_code = "import numpy as np\n" + self.indent_code(initial_code) + "\n"

            file_path = os.path.join(folder_name, "env", "GPTrewardfunc_0.py")
            with open(file_path, "w") as fp:
                fp.write(reward_code)
            rewardfunc_files.append(file_path)
            print(f"initial Saved: {file_path}")
        messages.append({"role": "assistant", "content": initial_code})

        # 生成不同的多样化 Reward Functions
        for i in range(1, rewardfunc_nums):
            diverse_messages = messages + [
                {"role": "user", "content": rewardfunc_div_prompts + zeroshot_rewardfunc_format}
            ]
            # print(diverse_messages)
            response = self.client.chat.completions.create(
                model=self.model, messages=diverse_messages, n=1
            )
            diverse_code = self.extract_code(response.choices[0].message.content)
            messages.append({"role": "assistant", "content": diverse_code})

            if diverse_code:
                reward_code =  "import numpy as np\n" + self.indent_code(diverse_code) + "\n"
                file_path = os.path.join(folder_name, "env", f"GPTrewardfunc_{i}.py")
                with open(file_path, "w") as fp:
                    fp.write(reward_code)
                rewardfunc_files.append(file_path)
                print(f"Saved: {file_path}")

        return rewardfunc_files


    def generate_morphology(self, morphology_nums, folder_name):
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=morphology_nums                                                                                                                                                                                                                                                                                   
        )

        # 解析所有 response 里的参数
        for i, choice in enumerate(responses.choices):
            print(f"Response {i}:")
            print(json.dumps(choice.message.content, indent=4))

        parameter_list = [json.loads(choice.message.content).get('parameters', []) for choice in responses.choices]
        material_list = [compute_hopper_volume(parameter) for parameter in parameter_list]

        xml_files = []
        for i, parameter in enumerate(parameter_list):
            if not isinstance(parameter, list):
                print(f"Skipping invalid parameter {i}: {parameter}")
                continue

            xml_file = hopper_design(parameter)  
            filename = f"GPTHopper_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            xml_files.append(file_path)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            print(f"Successfully saved {filename}")
            
        return xml_files, material_list, parameter_list
    
    def generate_morphology_div(self, morphology_nums, folder_name):

        material_list = []
        xml_files = []
        parameter_list = []
        
        # 生成初始 morphology
        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_prompts + morphology_format}
        ]
        
        response = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
            n=1
        )
        

        initial_parameter = json.loads(response.choices[0].message.content)
        parameter_list.append(initial_parameter['parameters'])
        material_list.append(compute_hopper_volume(initial_parameter['parameters']))
        messages.append({"role": "assistant", "content": json.dumps(initial_parameter)})

        logging.info(f"generate initial_parameter{initial_parameter['parameters']}" )

        xml_file = hopper_design(initial_parameter['parameters'])  

        filename = f"GPTHopper_0.xml"
        file_path = os.path.join(folder_name, "assets", filename)
        with open(file_path, "w") as fp:
            fp.write(xml_file)

        xml_files.append(file_path)

        # 生成不同的多样化设计
        for i in range(1, morphology_nums):
            diverse_messages = messages + [
                {"role": "user", "content": morphology_div_prompts + morphology_format}
            ]
            
            response = self.client.chat.completions.create(
                model=self.model,
                messages=diverse_messages,
                response_format={'type': 'json_object'},
                n=1
            )

            diverse_parameter = json.loads(response.choices[0].message.content)
            material_list.append(compute_hopper_volume(diverse_parameter['parameters'])) 
            parameter_list.append(diverse_parameter['parameters'])
            messages.append({"role": "assistant", "content": json.dumps(diverse_parameter)})
            logging.info(f"generate diverse_parameter{ diverse_parameter['parameters']}")
            xml_file = hopper_design(diverse_parameter['parameters'])  
            filename = f"GPTHopper_{i}.xml"
            file_path = os.path.join(folder_name, "assets", filename)
            with open(file_path, "w") as fp:
                fp.write(xml_file)
            xml_files.append(file_path)

        return xml_files, material_list, parameter_list


    def improve_rewardfunc(self, best_rewardfunc, rewardfunc_list, fitness_list, folder_name, step, rewardfunc_index, morphology_index):
        reward_improve_prompts = prompts.reward_improve_prompts

        for rewardfunc_file, fitness in zip(rewardfunc_list, fitness_list):
            with open(rewardfunc_file, "r") as fp:
                reward_content = fp.read()
            reward_improve_prompts += f"\nreward function:\n{reward_content}\nfitness: {fitness}\n"

        with open(best_rewardfunc, "r") as fp:
            best_reward_content = fp.read()
        reward_improve_prompts += f"\nbest reward function:\n{best_reward_content}\nbest fitness: {max(fitness_list)}\n"

        messages = [
            {"role": "system", "content": "You are a reinforcement learning reward function designer"},
            {"role": "user", "content":reward_improve_prompts+ zeroshot_rewardfunc_format}
        ]
        print(messages)
        response = self.client.chat.completions.create(
            model=self.model, messages=messages
        )

        print(response)
        reward_code = self.extract_code(response.choices[0].message.content)

        if reward_code:
            full_code = "import numpy as np \n" + self.indent_code(reward_code) + "\n"
            file_name =  f"GPTrewardfunc_refine_{step}_{rewardfunc_index}_{morphology_index}.py"
            file_path = os.path.join(folder_name, "env", file_name)
            with open(file_path, "w") as fp:
                fp.write(full_code)

        return file_path

    def improve_morphology(self, best_parameter, parameter_list, fitness_list, folder_name, rewardfunc_index, morphology_index, iteration):
        morphology_improve_prompts = prompts.morphology_improve_prompts
        for parameter_content, fitness in zip(parameter_list, fitness_list):
            morphology_improve_prompts = morphology_improve_prompts + f"parameter:{parameter_content} \n" + f"fintess:{fitness}"
        morphology_improve_prompts = morphology_improve_prompts + f"best parameter:{best_parameter} \n" + f"best fintess:{max(fitness_list)}"  

        messages = [
            {"role": "system", "content": "You are a helpful mujoco robot designer"},
            {"role": "user", "content": morphology_improve_prompts + morphology_format}
        ]
        
        responses = self.client.chat.completions.create(
            model=self.model,
            messages=messages,
            response_format={'type': 'json_object'},
        )
        # print(responses)
        parameter = json.loads(responses.choices[0].message.content).get('parameters', []) 
        print(parameter)
        xml_file = hopper_design(parameter)  
        filename = f"GPTHopper_refine_{rewardfunc_index}_{morphology_index}_{iteration}.xml"
        file_path = os.path.join(folder_name, "assets", filename)

        with open(file_path, "w") as fp:
            fp.write(xml_file)

        print(f"Successfully saved {filename}")
        return file_path, parameter


# Configuration

In [14]:
folder_name = "results/Div_m25_r5"
log_file = os.path.join(folder_name, "new_parameters.log")
logging.basicConfig(filename=log_file, level=logging.INFO, format="%(asctime)s - %(message)s")

# folder_name = setup_logging(div_flag=True)

best_fitness = float('-inf')  
best_morphology = None  
best_rewardfunc = None  
best_reward = None
best_material = None
best_efficiency = None

morphology_nums = 51
rewardfunc_nums = 11

fitness_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
efficiency_matrix = np.array([[None for _ in range(morphology_nums)] for _ in range(rewardfunc_nums)])
fitness_list = []
designer = DGA()


# print configuration info

In [15]:
logging.info(f"start!")

In [16]:
parameter_list = [[1.2, 0.9, 0.6, 0.3, 0.3, -0.3, 0.05, 0.04, 0.05, 0.05],
 [1.5, 1.2, 0.8, 0.5, 0.5, -0.5, 0.03, 0.03, 0.03, 0.04],
 [1.0, 0.7, 0.5, 0.2, 0.2, -0.2, 0.04, 0.05, 0.06, 0.07],
 [1.0, 0.6, 0.4, 0.1, 0.1, -0.1, 0.05, 0.04, 0.03, 0.02],
 [1.3, 0.9, 0.4, 0.1, 0.1, -0.1, 0.02, 0.02, 0.04, 0.06],
 [1.8, 1.3, 0.7, 0.2, 0.2, -0.2, 0.01, 0.03, 0.05, 0.06],
 [1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03],
 [1.0, 0.9, 0.6, 0.2, 0.2, -0.2, 0.02, 0.04, 0.06, 0.08],
 [1.5, 1.2, 0.8, 0.4, 0.35, -0.35, 0.02, 0.02, 0.01, 0.05],
 [1.2, 0.8, 0.5, 0.3, 0.25, -0.25, 0.03, 0.05, 0.06, 0.02],
 [1.6, 1.1, 0.7, 0.4, 0.35, -0.35, 0.03, 0.03, 0.03, 0.04],
 [1.4, 0.9, 0.6, 0.2, 0.2, -0.2, 0.01, 0.01, 0.02, 0.04],
 [1.8, 1.1, 0.9, 0.6, 0.55, -0.6, 0.015, 0.015, 0.015, 0.025],
 [1.7, 1.2, 0.8, 0.3, 0.25, -0.25, 0.01, 0.03, 0.04, 0.02],
 [1.0, 0.6, 0.3, 0.1, 0.08, -0.08, 0.015, 0.025, 0.035, 0.045],
 [1.2, 0.5, 0.2, 0.1, 0.12, -0.12, 0.02, 0.015, 0.02, 0.05],
 [1.0, 0.8, 0.2, 0.05, 0.05, -0.05, 0.01, 0.02, 0.03, 0.06],
 [1.8, 0.6, 0.3, 0.05, 0.05, -0.05, 0.025, 0.035, 0.045, 0.02],
 [0.9, 0.4, 0.15, 0.05, 0.05, -0.05, 0.02, 0.03, 0.04, 0.01],
 [1.5, 0.8, 0.5, 0.2, 0.2, -0.2, 0.01, 0.01, 0.01, 0.04],
 [1.4, 0.6, 0.3, 0.1, 0.12, -0.12, 0.015, 0.04, 0.06, 0.02],
 [2.0, 1.0, 0.6, 0.1, 0.15, -0.15, 0.01, 0.02, 0.03, 0.05],
 [0.8, 0.5, 0.4, 0.2, 0.25, -0.25, 0.01, 0.015, 0.02, 0.03],
 [1.1, 0.8, 0.6, 0.2, 0.19, -0.19, 0.015, 0.03, 0.05, 0.02],
 [1.2, 0.4, 0.3, 0.15, 0.15, -0.15, 0.02, 0.04, 0.05, 0.01],
 [1.5, 1.2, 0.9, 0.5, 0.5, -0.5, 0.015, 0.02, 0.025, 0.03],
 [1.0, 0.8, 0.5, 0.1, 0.1, -0.1, 0.02, 0.015, 0.01, 0.06],
 [1.5, 0.7, 0.4, 0.1, 0.1, -0.1, 0.015, 0.025, 0.03, 0.05],
 [1.5, 1.0, 0.4, 0.15, 0.2, -0.2, 0.015, 0.01, 0.035, 0.05],
 [1.5, 1.0, 0.6, 0.3, 0.3, -0.3, 0.015, 0.02, 0.025, 0.03],
 [1.6, 1.2, 0.7, 0.2, 0.25, -0.25, 0.01, 0.04, 0.06, 0.02],
 [1.4, 0.6, 0.4, 0.2, 0.2, -0.35, 0.012, 0.015, 0.018, 0.06],
 [1.0, 0.6, 0.3, 0.05, 0.06, -0.06, 0.01, 0.02, 0.03, 0.05],
 [1.8, 1.0, 0.8, 0.15, 0.15, -0.15, 0.02, 0.02, 0.05, 0.07],
 [1.0, 0.6, 0.25, 0.05, 0.1, -0.1, 0.01, 0.01, 0.04, 0.05],
 [1.0, 0.3, 0.15, 0.05, 0.05, -0.05, 0.015, 0.025, 0.05, 0.1],
 [1.8, 1.2, 0.8, 0.2, 0.15, -0.15, 0.015, 0.04, 0.06, 0.08],
 [1.5, 0.7, 0.3, 0.05, 0.05, -0.05, 0.015, 0.03, 0.07, 0.1],
 [1.8, 1.4, 0.9, 0.3, 0.3, -0.3, 0.012, 0.013, 0.018, 0.065],
 [1.5, 1.0, 0.6, 0.05, 0.05, -0.05, 0.01, 0.02, 0.03, 0.06],
 [1.8, 0.5, 0.2, 0.05, 0.05, -0.05, 0.01, 0.05, 0.1, 0.2],
 [1.2, 0.9, 0.5, 0.1, 0.07, -0.12, 0.02, 0.01, 0.02, 0.08],
 [1.5, 1.0, 0.6, 0.3, 0.35, -0.35, 0.014, 0.018, 0.1, 0.03],
 [1.0, 0.8, 0.5, 0.1, 0.1, -0.1, 0.05, 0.02, 0.01, 0.1],
 [1.2, 0.6, 0.4, 0.15, 0.15, -0.15, 0.015, 0.02, 0.03, 0.04],
 [1.8, 1.4, 1.0, 0.5, 0.6, -0.7, 0.015, 0.02, 0.025, 0.15],
 [1.5, 0.5, 0.2, 0.1, 0.1, -0.15, 0.012, 0.02, 0.04, 0.08],
 [1.4, 0.85, 0.4, 0.05, 0.05, -0.05, 0.015, 0.01, 0.02, 0.03],
 [1.6, 0.5, 0.25, 0.05, 0.05, -0.05, 0.008, 0.015, 0.03, 0.1],
 [1.5, 1.2, 0.6, 0.1, 0.1, -0.3, 0.01, 0.03, 0.05, 0.07],
[1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]]

morphology_list = [f'results/Div_m25_r5/assets/GPTHopper_{i}.xml' for i in range(0,51) ]
rewardfunc_list = [f'results/Div_m25_r5/env/GPTrewardfunc_{i}.py' for i in range(0,11)]

material_list = [compute_hopper_volume(parameter) for parameter in parameter_list]

# enter coarse optimization stage

In [17]:
efficiency_matrix = np.array([[None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 416.00392985310305],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 380.17372338336355],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 561.6139087406524],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 381.10650937501947],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 495.97501954740835],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [48.64317013054387, 203.81506929121974, 75.46462999365069,
        564.2513466097603, 16.057820351949477, 906.2202450244702,
        -11.725614708265494, 41.82336731444111, 151.2091580265195,
        269.03077069812673, 263.77577467841803, 335.3823298069768,
        462.14178803383237, 844.139122389248, 223.39054046172868,
        248.7655359599275, -4.435027776773188, 280.797706653486,
        1653.5922168599618, 428.12462169225836, 580.3848884995529,
        808.3142418156576, 381.8550909407698, 852.4473279544258,
        397.168803236307, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 94.19255287549959]],
      dtype=object)

# print coarse optimization info

# configuration of fine optimization

# enter fine optimization stage

In [18]:
efficiency_matrix_select = np.array([[None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 416.00392985310305],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 380.17372338336355],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 561.6139087406524],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 381.10650937501947],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 495.97501954740835],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None],
       [48.64317013054387, 203.81506929121974, 75.46462999365069,
        564.2513466097603, 16.057820351949477, 906.2202450244702,
        -11.725614708265494, 41.82336731444111, 151.2091580265195,
        269.03077069812673, 263.77577467841803, 335.3823298069768,
        462.14178803383237, 844.139122389248, 223.39054046172868,
        248.7655359599275, -4.435027776773188, 280.797706653486,
        1653.5922168599618, 428.12462169225836, 580.3848884995529,
        808.3142418156576, 381.8550909407698, 852.4473279544258,
        397.168803236307, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, None, None, None, None, None,
        None, None, None, None, None, None, 94.19255287549959]],
      dtype=object)

In [21]:
len(rewardfunc_list)

11

In [25]:
best_rewardfunc

'results/Div_m25_r5/env/GPTrewardfunc_10.py'

In [27]:
final_optimized_results = []  # 用来记录每个 coarse_best 的最优结果
coarse_best = [(10,50)]
for rewardfunc_index, morphology_index in coarse_best:
    
    morphology = morphology_list[morphology_index]
    parameter = parameter_list[morphology_index]
    rewardfunc = rewardfunc_list[rewardfunc_index]
    
    best_efficiency = efficiency_matrix_select[rewardfunc_index][morphology_index]
    best_fitness = fitness_matrix[rewardfunc_index][morphology_index]
    best_morphology = morphology
    best_parameter = parameter
    best_rewardfunc = rewardfunc
    best_material = compute_hopper_volume(parameter)
    
    
    logging.info(f"Initial morphology:{morphology}")
    logging.info(f"Initial parameter:{parameter}" )
    logging.info(f"Initial rewardfunc:{rewardfunc}" )
    logging.info(f"Initial fitness:{best_fitness}" )
    logging.info(f"Initial efficiency:{best_efficiency}" )
    iteration = 0
    print(f"Initial parameter:{parameter}")
    while True:
        improved = False  # 标记是否有改进，方便控制循环

        designer = DGA()
        iteration +=1   
         # -------- 优化 morphology --------
        improved_morphology, improved_parameter = designer.improve_morphology(
            best_parameter,
            parameter_list[morphology_index],  # 这本身已经是list结构，可以保留
            [efficiency_matrix_select[rewardfunc_index, morphology_index]],  # 👈 用 [] 包装成列表
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )
        
        print("improved parameter", improved_parameter)
        shutil.copy(improved_morphology, "GPTHopper.xml")
        shutil.copy(best_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTHopperEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue
        improved_material = compute_hopper_volume(improved_parameter)
        improved_efficiency = improved_fitness / improved_material

        if improved_efficiency > best_efficiency:

            best_fitness = improved_fitness
            best_morphology = improved_morphology
            best_parameter = improved_parameter
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True
            iteration +=1
            logging.info(f"Morphology optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")

        # -------- 没有进一步改进，跳出循环 --------
        if not improved:
            logging.info("Not improved Morphology!")
            logging.info("____________________________________________")
            improved = False
            # break
            
        iteration +=1        
        # -------- 优化 reward function --------
        improved_rewardfunc = designer.improve_rewardfunc(
            best_rewardfunc,
            [rewardfunc_list[rewardfunc_index]],
            [efficiency_matrix_select[rewardfunc_index, morphology_index]],
            folder_name,
            rewardfunc_index, 
            morphology_index,
            iteration
        )
        
        shutil.copy(best_morphology, "GPTHopper.xml")
        shutil.copy(improved_rewardfunc, "GPTrewardfunc.py")
        
        import GPTrewardfunc
        importlib.reload(GPTrewardfunc)  # 重新加载模块
        from GPTrewardfunc import _get_rew
        GPTHopperEnv._get_rew = _get_rew
        try:
            model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=5e5)
            improved_fitness, _ = Eva(model_path)
            improved_material = compute_hopper_volume(best_parameter)
            improved_efficiency = improved_fitness / improved_material
            print("improved_fitness", improved_fitness)
        except Exception as e:
            print(f"Error evaluating design: {e}")
            continue

        if improved_efficiency > best_efficiency:
            best_fitness = improved_fitness
            best_rewardfunc = improved_rewardfunc
            best_material = improved_material
            best_efficiency = improved_efficiency
            improved = True

            logging.info(f"Reward optimization improved iteration {iteration}: material={improved_material}, fitness={improved_fitness}, efficiency={improved_efficiency}")
        
        if not improved:
            logging.info("Not improved Reward!")
            logging.info("____________________________________________")
            break
            
    # 保存当前 coarse_best 的最终最优结果
    final_optimized_results.append({
        "best_morphology": best_morphology,
        "best_parameter": best_parameter,
        "best_rewardfunc": best_rewardfunc,
        "best_fitness": best_fitness,
        "best_material": best_material,
        "best_efficiency": best_efficiency,
        "best_iteration":iteration
    })
    logging.info("____________________________________________")
    logging.info(f"Final optimized result: rewardfunc_index{rewardfunc_index} morphology_index{morphology_index}")
    logging.info(f"  Morphology: {best_morphology}")
    logging.info(f"  Parameter: {best_parameter}")
    logging.info(f"  Rewardfunc: {best_rewardfunc}")
    logging.info(f"  Fitness: {best_fitness}")
    logging.info(f"  Material: {best_material}")
    logging.info(f"  Efficiency: {best_efficiency}")
    logging.info("____________________________________________")

Initial parameter:[1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]
[1.5, 1.1, 0.65, 0.15, -0.12, 0.3, 0.04, 0.04, 0.03, 0.05]
Successfully saved GPTHopper_refine_10_50_1.xml
improved parameter [1.5, 1.1, 0.65, 0.15, -0.12, 0.3, 0.04, 0.04, 0.03, 0.05]
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
[{'role': 'system', 'content': 'You are a reinforcement learning reward function designer'}, {'role': 'user', 'content': '\nYou are a reward engineer trying to write reward functions to solve reinforcement learning tasks as effectively as possible.\nYour goal is to write a reward function for the enviroment that will help the agent learn the task described in text.\n\nTask Description: The hopper is a two-dimensional one-legged figure consisti

In [ ]:

logging.info(f"{final_optimized_results}")

# logging.info(f"fine optimization end: best material cost: {best_material}  fitness: {improved_fitness} merterial_efficiency: {improved_material_efficiency}")

In [29]:
final_optimized_results

[{'best_morphology': 'results/Div_m25_r5/assets/GPTHopper_refine_10_50_12.xml',
  'best_parameter': [1.6, 1.2, 0.75, 0.25, -0.1, 0.4, 0.04, 0.02, 0.02, 0.03],
  'best_rewardfunc': 'results/Div_m25_r5/env/GPTrewardfunc_refine_10_50_5.py',
  'best_fitness': 9.248461495176302,
  'best_material': 0.00506634175268914,
  'best_efficiency': 1825.4713058524633,
  'best_iteration': 16}]

In [8]:
# robodesign best

best_morphology = "results/Div_m25_r5/assets/GPTHopper_refine_1_13_0.xml"
best_rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_1.py"
morphology_index=999
rewardfunc_index=999
best_parameter = [1.75, 1.15, 0.75, 0.25, 0.2, -0.2, 0.01, 0.025, 0.035, 0.025]

shutil.copy(best_morphology, "GPTHopper.xml")
shutil.copy(best_rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew


model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
best_fitness, _ = Eva(model_path)
best_material = compute_hopper_volume(best_parameter)
best_efficiency = best_fitness / best_material

logging.info("3e6 steps train\n")
logging.info(f"best_fitness:{best_fitness}")
logging.info(f"best_efficiency:{best_efficiency}")
print("3e6 steps train\n")
print(f"best_fitness:{best_fitness}")
print(f"best_efficiency:{best_efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
3e6 steps train

best_fitness:15.099855627034716
best_efficiency:3776.6631687007007


2025-04-07 02:15:56,188 - Initial morphology:results/Div_m25_r5/assets/GPTHopper_6.xml
2025-04-07 02:15:56,188 - Initial parameter:[1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03]
2025-04-07 02:15:56,188 - Initial rewardfunc:results/Div_m25_r5/env/GPTrewardfunc_2.py
2025-04-07 02:15:56,188 - Initial fitness:5.2437351113867665
2025-04-07 02:15:56,188 - Initial efficiency:1951.4412234802828

In [9]:
# coarse only best

morphology = "results/Div_m25_r5/assets/GPTHopper_6.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=777
rewardfunc_index=777
parameter = [1.0, 0.8, 0.5, 0.2, 0.18, -0.2, 0.03, 0.02, 0.02, 0.03]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块rewardfunc
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("coarse only best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
3e6 steps train

fitness:7.887110028279259
efficiency:2935.1657389952516


In [10]:
# human


morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=888
rewardfunc_index=888
parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("human best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("human 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
human 3e6 steps train

fitness:6.8610045928497785
efficiency:433.6914525166767


2025-04-07 10:50:19,543 - morphology: 50, rewardfunc: 2, material cost: 0.005429560128926578 reward: 1016.4147318163663 fitness: 3.0493164867488556 efficiency: 561.6139087406524

In [15]:
# Robodesign (w/o Morphology Design)

morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=555
rewardfunc_index=555

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Morphology Design) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Morphology Design) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Morphology Design) 3e6 steps train

fitness:4.233495359427274
efficiency:267.60377825224765


2025-04-07 09:35:14,008 - morphology: 18, rewardfunc: 10, material cost: 0.0022881266493645657 reward: 1471.1455354782527 fitness: 3.783628418579109 efficiency: 1653.5922168599618

In [20]:
# Robodesign (w/o Reward Shaping)

morphology = "results/Div_m25_r5/assets/GPTHopper_18.xml"
rewardfunc = "results/Div_m25_r5/env/GPTrewardfunc_10.py"

morphology_index=444
rewardfunc_index=444

parameter =  [0.9, 0.4, 0.15, 0.05, 0.05, -0.05, 0.02, 0.03, 0.04, 0.01]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Reward Shaping) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Reward Shaping) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Reward Shaping) 3e6 steps train

fitness:4.457903414345486
efficiency:1948.2765150187327


2025-04-08 00:50:43,757 - Initial morphology:results/noDiv_m25_r5/assets/GPTHopper_16.xml
2025-04-08 00:50:43,757 - Initial parameter:[1.5, 1.1, 0.7, 0.3, 0.4, -0.1, 0.06, 0.04, 0.03, 0.02]
2025-04-08 00:50:43,757 - Initial rewardfunc:results/noDiv_m25_r5/env/GPTrewardfunc_2.py
2025-04-08 00:50:43,757 - Initial fitness:1.0491079684899782
2025-04-08 00:50:43,757 - Initial efficiency:1222.0645575053077

In [17]:
# Robodesign (w/o Diversity Reflection)

morphology = "results/noDiv_m25_r5/assets/GPTHopper_16.xml"
rewardfunc = "results/noDiv_m25_r5/env/GPTrewardfunc_2.py"

morphology_index=333
rewardfunc_index=333

parameter =  [1.5, 1.1, 0.7, 0.3, 0.4, -0.1, 0.06, 0.04, 0.03, 0.02]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info("Robodesign (w/o Diversity Reflection) best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print("Robodesign (w/o Diversity Reflection) 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
Robodesign (w/o Diversity Reflection) 3e6 steps train

fitness:4.726470799983718
efficiency:491.66090927902854


2025-04-09 04:00:22,222 - iteration:2, morphology: 0, rewardfunc: 2, material cost: 0.015820013405927 reward: 1428.1315493516208 fitness: 3.4351751639953476 efficiency: 217.14110322487792

In [18]:
# eureka reward

morphology = "results/Div_m25_r5/assets/GPTHopper_50.xml"
rewardfunc = "results/eureka/env/GPTrewardfunc_2_2.py"


morphology_index=222
rewardfunc_index=222

parameter = [1.45, 1.06, 0.6, 0.1, -0.13, 0.26, 0.05, 0.05, 0.04, 0.06]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info(" eureka reward best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" eureka reward 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 eureka reward 3e6 steps train

fitness:6.7727740674190535
efficiency:428.1143064570109


In [21]:

morphology = "results/eureka_morphology/assets/GPTHopper_6.xml"
rewardfunc = "results/eureka_morphology/env/GPTrewardfunc_0.py"

morphology_index=111
rewardfunc_index=111

parameter = [0.7, 0.42, 0.22, 0.09, -0.05, 0.09, 0.035, 0.035, 0.025, 0.015]

shutil.copy(morphology, "GPTHopper.xml")
shutil.copy(rewardfunc, "GPTrewardfunc.py")

import GPTrewardfunc
importlib.reload(GPTrewardfunc)  # 重新加载模块
from GPTrewardfunc import _get_rew
GPTHopperEnv._get_rew = _get_rew

model_path = Train(morphology_index, rewardfunc_index, folder_name, stage='fine', total_timesteps=1e6)
# model_path = f"results/Div_m25_r5/fine/SAC_morphology{morphology_index}_rewardfunc{rewardfunc_index}_3000000.0steps"
fitness, _ = Eva(model_path)
material = compute_hopper_volume(parameter)
efficiency = fitness / material

logging.info(" eureka reward best 3e6 steps train\n")
logging.info(f"fitness:{fitness}")
logging.info(f"efficiency:{efficiency}")
print(" eureka reward 3e6 steps train\n")
print(f"fitness:{fitness}")
print(f"efficiency:{efficiency}")


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
 eureka reward 3e6 steps train

fitness:1.608018014941748
efficiency:609.0407908588114
